### Данный Jupiter-ноутбук предназначен для запуска на Jupyter-сервере внутри ML Space. Если вы хотите выполнить те же самые действия из удаленного Jupyter-сервера, посмотрите пример отправки задач на обучение через Training Job API.

# 0. Подключение библиотеки для работы с кластером и сервисом

In [15]:
try:
    import client_lib
except ImportError:
    raise RuntimeError("Скрипт не предназначен для запуска вне кластера")

In [16]:
import requests

In [17]:
def save_file(url, filename):
    # Download file and place it on local storage
    r = requests.get(url)

    with open(filename, 'wb') as f:
        f.write(r.content)
    print(f"{filename} downloaded from {url}")

# 1. Размещаем модель и данные на локальные диски кластера

In [18]:
save_file("https://github.com/sbercloud-ai/aicloud-examples/raw/master/quick-start/job_launch_tf2/mnist.npz", "mnist.npz")
save_file("https://github.com/sbercloud-ai/aicloud-examples/raw/master/quick-start/job_launch_tf2/requirements.txt", "requirements.txt")
save_file("https://raw.githubusercontent.com/sbercloud-ai/aicloud-examples/master/quick-start/job_launch_tf2/tensorflow_mnist_estimator.py", "tensorflow_mnist_estimator.py")

mnist.npz downloaded from https://github.com/sbercloud-ai/aicloud-examples/raw/master/quick-start/job_launch_tf2/mnist.npz
requirements.txt downloaded from https://github.com/sbercloud-ai/aicloud-examples/raw/master/quick-start/job_launch_tf2/requirements.txt
tensorflow_mnist_estimator.py downloaded from https://raw.githubusercontent.com/sbercloud-ai/aicloud-examples/master/quick-start/job_launch_tf2/tensorflow_mnist_estimator.py


# 2. Сборка кастомного образа с нужными библиотеками

##### Содержимое файла requirements.txt

In [5]:
%cat /home/jovyan/quick-start/job_launch_tf2/requirements.txt

# tensorflow-gpu 2.3.0 already installed in registry.aicloud.sbcp.ru/base/horovod-cuda10.1-tf2.3.0
# with other libraries, we add one more:
requests

##### Запуск сборки кастомного образа с необходимыми библиотеками

После выполнения этой задачи собранный образ должен оказаться в docker registry

In [7]:
job = client_lib.ImageBuildJob(
    # Один из доступных образов с постфиксом a100
    from_image='cr.msk.sbercloud.ru/aicloud-base-images/horovod-cuda11.0-tf2.4.0-pt1.7.1-a100',
    # файл с зависимостями для кастомного образа
    requirements_file='/home/jovyan/quick-start/job_launch_tf2/requirements.txt'
)

job.submit()

'ImageBuildJob "{\'image\': \'cr.msk.sbercloud.ru/eec0971f-da40-49ba-a962-d08ec3a53a0a/job-custom-image-0ac4af\', \'name\': \'image-build-job-nmsnt\', \'status\': \'ok\'}" created'

In [8]:
job.new_image # идентификатор кастомного образа

'cr.msk.sbercloud.ru/eec0971f-da40-49ba-a962-d08ec3a53a0a/job-custom-image-0ac4af'

In [ ]:
job.logs() # просмотр логов сборки образа в интерактивном режиме

INFO[0006] Resolved base name cr.msk.sbercloud.ru/aicloud-base-images-test/horovod-cuda11.0-tf2.4.0-pt1.7.1-a100 to cr.msk.sbercloud.ru/aicloud-base-images-test/horovod-cuda11.0-tf2.4.0-pt1.7.1-a100 
INFO[0006] Resolved base name cr.msk.sbercloud.ru/aicloud-base-images-test/horovod-cuda11.0-tf2.4.0-pt1.7.1-a100 to cr.msk.sbercloud.ru/aicloud-base-images-test/horovod-cuda11.0-tf2.4.0-pt1.7.1-a100 
INFO[0006] Retrieving image manifest cr.msk.sbercloud.ru/aicloud-base-images-test/horovod-cuda11.0-tf2.4.0-pt1.7.1-a100 
INFO[0008] Retrieving image manifest cr.msk.sbercloud.ru/aicloud-base-images-test/horovod-cuda11.0-tf2.4.0-pt1.7.1-a100 
INFO[0011] Built cross stage deps: map[]                
INFO[0011] Retrieving image manifest cr.msk.sbercloud.ru/aicloud-base-images-test/horovod-cuda11.0-tf2.4.0-pt1.7.1-a100 
INFO[0012] Retrieving image manifest cr.msk.sbercloud.ru/aicloud-base-images-test/horovod-cuda11.0-tf2.4.0-pt1.7.1-a100 
INFO[0015] Unpacking rootfs as cmd RUN if [ -e /context/req

# 3. Запуск задачи обучения на кластере

#### Создание задачи обучения и отправка ее на кластер

Параметр `base_image` предназначен для запуска задачи на кластере в своем кастомном образе (см. пункт 2 выше). Переменная `job.new_image` содержит название собранного кастомного образа (вида: `cr.msk.sbercloud.ru/eec0971f-da40-49ba-a962-d08ec3a53a0a/job-custom-image-0ac4af`).

По окончании сборки кастомного образа его можно указать в поле `base_image` и отправить задачу обучения на кластер с использованием этого образа:

```python
mnist_tf_run = client_lib.Job(base_image=job.new_image, # кастомный образ
                              script='/home/jovyan/quick-start/job_launch_tf2/tensorflow_mnist_estimator.py',
                              n_workers=2, instance_type=a100.1gpu.80vG.12C.96G)
```

In [19]:
# базовый образ, можно заменить на свой (смотри выше)
mnist_tf_run = client_lib.Job(
    base_image='cr.msk.sbercloud.ru/aicloud-base-images/horovod-cuda11.0-tf2.4.0-pt1.7.1-a100',
    script='/home/jovyan/quick-start/job_launch_tf2/tensorflow_mnist_estimator.py',
    n_workers=2, instance_type=a100.2gpu,
    region='A100-MT' # Если не указать регион, то задача автоматически отправится на V100
)

В предыдущей строке мы указали образ, в рамках которого будет исполняться задача (`base_image`), скрипт, который будет запущен (`script`), а также количество рабочих узлов кластера (`n_workers`) и и конфигурация вычислительных ресурсов (instance_type). 
В этом случае под задачу будет запущено 2 рабочих узла с конфигурацией `a100.2gpu` (ИТОГО - 4 GPU).

Теперь вызовем на объекте метод `.submit()` и отправим задачу в очередь исполнения на кластере.

In [20]:
mnist_tf_run.submit()

'Job "lm-mpi-job-def8c2d8-9b83-4a80-806d-aa685bedb3ac" created'

#### Мониторинг задач

Вы можете просмотреть список ваших задач и их статус с помощью следующего метода:

In [32]:
client_lib.jobs(region='A100')

2021-11-01T07:56:25Z : lm-mpi-job-0b40ee2f-7242-4bec-bccc-66e293cb8e7c : Completed
2021-11-12T15:17:09Z : lm-mpi-job-1caf677e-3450-4d82-a256-195490778c90 : Completed
2021-11-02T07:59:12Z : lm-mpi-job-1efc80c5-75df-4756-ad9f-a0d11916dd67 : Completed
2021-11-15T12:58:18Z : lm-mpi-job-22647b15-feb4-4540-8ee3-e7af29d29db8 : Completed
2021-11-05T13:35:26Z : lm-mpi-job-2ba6a7d9-09f5-4de3-81db-8b06d78bdac3 : Completed
2021-11-09T11:24:25Z : lm-mpi-job-2fe3bd66-b7ab-4e97-869d-3d102ae15e35 : Completed
2021-11-15T13:28:51Z : lm-mpi-job-40e2df84-3c0d-4c69-83ba-c7bd67a26451 : Completed
2021-10-29T21:11:03Z : lm-mpi-job-4b35717c-3b18-4d68-82c7-0549b25461de : Completed
2021-11-15T10:36:36Z : lm-mpi-job-571b056e-211b-45d0-a8b9-4d65fb56d317 : Completed
2021-11-15T08:00:06Z : lm-mpi-job-5b5aa221-d5d1-4267-9b05-e0a4de0f3967 : Completed
2021-11-15T12:58:20Z : lm-mpi-job-5dd2dd82-b73a-4f97-b514-01a31f58430e : Completed
2021-11-15T10:57:17Z : lm-mpi-job-645a1598-9c8b-4075-8b12-7df9affa89bc : Completed
2021

Как можно получить логи:

In [33]:
mnist_tf_run.logs()

2021-11-15T16:04:33.748518999Z lm-mpi-job-def8c2d8-9b83-4a80-806d-aa685bedb3ac-mpiworker-0.lm-mpi-job-def8c2d8-9b83-4a80-806d-aa685bedb3ac
2021-11-15T16:04:33.748769366Z 1
2021-11-15T16:04:33.750164032Z worker name is lm-mpi-job-def8c2d8-9b83-4a80-806d-aa685bedb3ac-mpiworker-0.lm-mpi-job-def8c2d8-9b83-4a80-806d-aa685bedb3ac
2021-11-15T16:04:33.752451078Z lm-mpi-job-def8c2d8-9b83-4a80-806d-aa685bedb3ac-mpiworker-0.lm-mpi-job-def8c2d8-9b83-4a80-806d-aa685bedb3ac is running..
2021-11-15T16:04:33.753757844Z lm-mpi-job-def8c2d8-9b83-4a80-806d-aa685bedb3ac-mpiworker-0.lm-mpi-job-def8c2d8-9b83-4a80-806d-aa685bedb3ac:1,
2021-11-15T16:04:33.756731213Z cat: /home/jovyan/.ssh/ai0001011-00848.pub: No such file or directory
2021-11-15T16:04:33.963496444Z Warning: Permanently added '[lm-mpi-job-def8c2d8-9b83-4a80-806d-aa685bedb3ac-mpiworker-0.lm-mpi-job-def8c2d8-9b83-4a80-806d-aa685bedb3ac]:2222,[10.233.113.102]:2222' (ECDSA) to the list of known hosts.
2021-11-15T16:04:34.388470634Z 2021-11-15 19:0

In [35]:
client_lib.logs("lm-mpi-job-def8c2d8-9b83-4a80-806d-aa685bedb3ac", region='A100')

2021-11-15T16:04:33.748518999Z lm-mpi-job-def8c2d8-9b83-4a80-806d-aa685bedb3ac-mpiworker-0.lm-mpi-job-def8c2d8-9b83-4a80-806d-aa685bedb3ac
2021-11-15T16:04:33.748769366Z 1
2021-11-15T16:04:33.750164032Z worker name is lm-mpi-job-def8c2d8-9b83-4a80-806d-aa685bedb3ac-mpiworker-0.lm-mpi-job-def8c2d8-9b83-4a80-806d-aa685bedb3ac
2021-11-15T16:04:33.752451078Z lm-mpi-job-def8c2d8-9b83-4a80-806d-aa685bedb3ac-mpiworker-0.lm-mpi-job-def8c2d8-9b83-4a80-806d-aa685bedb3ac is running..
2021-11-15T16:04:33.753757844Z lm-mpi-job-def8c2d8-9b83-4a80-806d-aa685bedb3ac-mpiworker-0.lm-mpi-job-def8c2d8-9b83-4a80-806d-aa685bedb3ac:1,
2021-11-15T16:04:33.756731213Z cat: /home/jovyan/.ssh/ai0001011-00848.pub: No such file or directory
2021-11-15T16:04:33.963496444Z Warning: Permanently added '[lm-mpi-job-def8c2d8-9b83-4a80-806d-aa685bedb3ac-mpiworker-0.lm-mpi-job-def8c2d8-9b83-4a80-806d-aa685bedb3ac]:2222,[10.233.113.102]:2222' (ECDSA) to the list of known hosts.
2021-11-15T16:04:34.388470634Z 2021-11-15 19:0

(ЕСЛИ НЕОБХОДИМО ПРЕРВАТЬ ЗАДАЧУ) Варианты остановки задачи:

In [31]:
mnist_tf_run.kill()

'Job "lm-mpi-job-fe7e0199-a1ab-46b8-8097-1ed1a9cced65" deleted'

In [78]:
client_lib.kill('lm-mpi-job-797862d6-cbba-4d6e-8f94-9c6e8094e527', region='A100')

'Job "lm-mpi-job-f370676b-980d-46b5-91b1-96ddc69e2d17" deleted'

# 4. Сохранение промежуточных результатов обучения модели

Если в процессе обучения модели пользователь сохраняет промежуточные результаты (checkpoints) обучения, они попадают в папку `/home/jovyan/quick-start/job_launch_tf2/checkpoints_tf/mnist_convnet_model/`. Их можно скачать через веб-интерфейс Jupyter-ноутбука или скопировать из локально доступной файловой системы в хранилище S3. Внутри кода задачи обучения можно сохранять метрики модели с помощью `mlflow` (пример — в коде `tensorflow_mnist_estimator.py`).

## Выгрузка результатов обучения модели с NFS на S3

##### Указываем параметры доступа к бакету S3

Для перемещения данных из локального хранилища `/home/jovyan` в объектное хранилище S3 необходимо указать параметры доступа к бакету S3. Эти данные можно найти на портале portal.sbercloud.ru в параметрах заказанной услуги AI Cloud ("Мои услуги"->"AI Cloud").

В одинарных кавычках `''` введите  по порядку: `S3 namespace`, `S3 access key`, `S3 security key`

In [10]:
client_lib.save_aws_credentials('', '', '')

'S3 Credentials "{\'result\': None, \'status\': \'ok\'}" created'

Ниже укажем бакет s3, в который будут копироваться файлы

In [11]:
s3_bucket = ''

In [13]:
# Перемещение отдельных файлов
client_lib.S3CopyJob("/home/jovyan/quick-start/job_launch_tf2/checkpoints_tf/mnist_convnet_model/checkpoint.data-00000-of-00001", s3_bucket).submit()
client_lib.S3CopyJob("/home/jovyan/quick-start/job_launch_tf2/checkpoints_tf/mnist_convnet_model/checkpoint.ckpt.index", s3_bucket).submit()
client_lib.S3CopyJob("/home/jovyan/quick-start/job_launch_tf2/checkpoints_tf/mnist_convnet_model/checkpoint", s3_bucket).submit()

'S3CopyJob "{\'name\': \'copying-job-mjq42\', \'status\': \'ok\'}" created'

In [14]:
# Перемещение папки с файлами
client_lib.S3CopyJob("/home/jovyan/quick-start/job_launch_tf2/checkpoints_tf/mnist_convnet_model/",
                     s3_bucket, recursive=True).submit()

'S3CopyJob "{\'name\': \'copying-job-w78wb\', \'status\': \'ok\'}" created'

Если все было сделано правильно, то промежуточные результаты обучения модели сохранятся в объектном хранилище S3, из которого можно будет впоследствии восстановить объект с моделью